## Import

In [489]:
import pandas as pd, optuna
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False
# import warnings
# def ignore_warn(*args, **kwargs):
#     pass
# warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from subprocess import check_output

In [490]:
import pandas as pd
from xgboost import plot_importance
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [491]:
kfold = KFold(n_splits=5,random_state=123, shuffle=True)

## 데이터 불러오기

In [492]:
train = pd.read_csv('open/train.csv')
test = pd.read_csv('open/test.csv')


In [493]:
train_ID = train["ID"]
test_ID = test["ID"]

In [494]:
# train.drop("ID", axis = 1, inplace = True)
# test.drop("ID", axis = 1, inplace = True)

In [495]:
X_train = train.drop(['ID','착과량(int)'], axis=1)
y_train = train['착과량(int)']

In [496]:
test = test.drop(['ID'], axis=1)

In [497]:
# y_train

In [498]:
y_train = np.log1p(y_train)


In [499]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
test = scaler.transform(test)

In [500]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import AdaBoostRegressor

In [501]:
#Validation function
n_folds = 5
def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train)
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [502]:
# lasso = make_pipeline(Lasso(alpha =0.00015440851710483001, random_state=1))
# ENet = make_pipeline(ElasticNet(alpha=0.00015440851710483001, l1_ratio=.8, random_state=3))
# Ennet = ElasticNet(random_state=1234)
Krr = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
Dec = DecisionTreeRegressor(max_depth = 7, min_samples_leaf =30)
Rf = RandomForestRegressor(n_estimators = 85, max_depth=75, min_samples_split=17, min_samples_leaf=14,
                           max_leaf_nodes=272)
Ada = AdaBoostRegressor(n_estimators = 389, learning_rate=0.0005173575102039275)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4,
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
Xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, random_state =7, nthread = -1)
Lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [503]:
# score = rmsle_cv(lasso)
# print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
# score = rmsle_cv(ENet)
# print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(Krr)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(Xgb) 
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(Lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))
score = rmsle_cv(Dec)
print("Dec score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))
score = rmsle_cv(Rf)
print("Rf score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))
score = rmsle_cv(Ada)
print("Ada score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))
# score = rmsle_cv(Ennet)
# print("Ennet score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))


Kernel Ridge score: 0.3772 (0.0373)

Gradient Boosting score: 0.3353 (0.0497)

Xgboost score: 0.3469 (0.0420)

[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_s

In [504]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [505]:
averaged_models = AveragingModels(models = (GBoost, Krr, Dec))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged base models score: 0.3242 (0.0455)



In [506]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for X_train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[X_train_index], y[X_train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [507]:
stacked_averaged_models = StackingAveragedModels(base_models = (Dec, GBoost, Krr),
                                                 meta_model = Rf)
score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

Stacking Averaged models score: nan (nan)


c:\Users\dlwjd\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
4 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\dlwjd\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\dlwjd\AppData\Local\Temp\ipykernel_46152\2459373258.py", line 20, in fit
    instance.fit(X[X_train_index], y[X_train_index])
  File "c:\Users\dlwjd\anaconda3\lib\site-packages\pandas\core\series.py", line 984, in __getitem__
    return self._get_with(key)
  File "c:\Users\dlwjd\anaconda3\lib\si

In [508]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [509]:
stacked_averaged_models.fit(X_train, y_train)
stacked_train_pred = stacked_averaged_models.predict(X_train)
stacked_pred = np.expm1(stacked_averaged_models.predict(test))
print(rmsle(y_train, stacked_train_pred))

0.30758219993992036


In [510]:
stacked_pred

array([243.24060706, 757.65709284, 148.22626546, ..., 360.3557267 ,
       234.09963463,  56.31406975])

In [511]:
Xgb.fit(X_train, y_train)
xgb_train_pred = Xgb.predict(X_train)
xgb_pred = np.expm1(Xgb.predict(test))
print(rmsle(y_train, xgb_train_pred))

0.08926252437113515


In [512]:
Lgb.fit(X_train, y_train)
lgb_train_pred = Lgb.predict(X_train)
lgb_pred = np.expm1(Lgb.predict(test))
print(rmsle(y_train, lgb_train_pred))

[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
0.16856442246805056


In [513]:
Ada.fit(X_train, y_train)
Ada_train_pred = Ada.predict(X_train)
Ada_pred = np.expm1(Ada.predict(test))
print(rmsle(y_train, Ada_train_pred))

0.3067081316569852


In [514]:
print('RMSLE score on train data:')
print(rmsle(y_train,stacked_train_pred*0.55 +
               xgb_train_pred*0.15 + lgb_train_pred*0.15 + Ada_train_pred * 0.15 ))

RMSLE score on train data:
0.2429067107563165


In [515]:
ensemble = stacked_train_pred*0.35 + xgb_train_pred*0.35 + lgb_train_pred*0.15 + Ada_train_pred * 0.15

In [516]:
# ensemble = ensemble.round(1)
# type(ensemble)
# ensemble = ensemble.astype(float)
ensemble =ensemble.astype(np.float)
ensemble =np.ceil(ensemble)

C:\Users\dlwjd\AppData\Local\Temp\ipykernel_46152\3865354930.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ensemble =ensemble.astype(np.float)


In [517]:
ensemble

array([7., 7., 7., ..., 7., 6., 3.])

## 모델 선언 및 학습

In [518]:
# from sklearn.neural_network import MLPRegressor

# model = MLPRegressor()
# # from sklearn.tree import DecisionTreeRegressor

# # model = DecisionTreeRegressor()

# model.fit(X_train, y_train)

## 추론

In [519]:
# sub = pd.DataFrame()
# sub['ID'] = test_ID
# sub['착과량(int)'] = ensemble
sample_submission = pd.read_csv('open/sample_submission.csv')


In [520]:
sample_submission['착과량(int)'] = ensemble
sample_submission.to_csv('./submit.csv', index=False, encoding='UTF-8')

ValueError: Length of values (2207) does not match length of index (2208)

## Submission

In [ ]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2208 entries, 0 to 2207
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        2208 non-null   object 
 1   착과량(int)  2208 non-null   float64
dtypes: float64(1), object(1)
memory usage: 34.6+ KB
